In [69]:
import pandas as pd
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.svm import SVR
import sklearn.datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras import layers, models


In [25]:
house_price_dataframe= pd.read_csv('清洗数据.csv')
house_price_dataframe =house_price_dataframe.drop('label',axis=1)
X = house_price_dataframe.drop(['Price'], axis=1)
Y = house_price_dataframe['Price']

In [27]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [29]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

In [93]:
X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=5)

In [43]:
ridge_regressor_cv = RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0], store_cv_values=True)
ridge_regressor_cv.fit(X_train, Y_train)
print(f"最佳的 alpha 值: {ridge_regressor_cv.alpha_}")

最佳的 alpha 值: 100.0


In [47]:
Y_train_pred = ridge_regressor_cv.predict(X_train)
Y_test_pred = ridge_regressor_cv.predict(X_test)

In [49]:
r2_train = r2_score(Y_train, Y_train_pred)
mse_train = mean_squared_error(Y_train, Y_train_pred)

# 测试集上的评估
r2_test = r2_score(Y_test, Y_test_pred)
mse_test = mean_squared_error(Y_test, Y_test_pred)

# 输出结果
print(f"训练集 R^2: {r2_train}")
print(f"训练集 MSE: {mse_train}")
print(f"测试集 R^2: {r2_test}")
print(f"测试集 MSE: {mse_test}")

训练集 R^2: 0.501729347554318
训练集 MSE: 0.6173983786219008
测试集 R^2: 0.5005253817279409
测试集 MSE: 0.595546384696465


In [75]:
svm_regressor = SVR(kernel='rbf', C=1.0, epsilon=0.1)  
svm_regressor.fit(X_train, Y_train)

SVR()

In [59]:
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 0.5]
}

In [61]:
grid_search = GridSearchCV(svm_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 0.5]},
             scoring='neg_mean_squared_error')

In [77]:
print(f"最佳参数组合: {grid_search.best_params_}")


最佳参数组合: {'C': 10, 'epsilon': 0.5}


In [79]:
best_svm_regressor = grid_search.best_estimator_

In [81]:
Y_train_pred =best_svm_regressor.predict(X_train)
Y_test_pred = best_svm_regressor.predict(X_test)
r2_train = r2_score(Y_train, Y_train_pred)
mse_train = mean_squared_error(Y_train, Y_train_pred)

# 测试集上的评估
r2_test = r2_score(Y_test, Y_test_pred)
mse_test = mean_squared_error(Y_test, Y_test_pred)

# 输出结果
print(f"训练集 R^2: {r2_train}")
print(f"训练集 MSE: {mse_train}")
print(f"测试集 R^2: {r2_test}")
print(f"测试集 MSE: {mse_test}")

训练集 R^2: 0.699599231950103
训练集 MSE: 0.37222129423124795
测试集 R^2: 0.6733965734278997
测试集 MSE: 0.38942417253832384


In [87]:
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1)) 
model.compile(optimizer='adam', loss='mean_squared_error')

In [89]:
history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test))


Epoch 1/100
508/508 [==============================] - 1s 1ms/step - loss: 0.7639 - val_loss: 0.4736
Epoch 2/100
508/508 [==============================] - 1s 1ms/step - loss: 0.5486 - val_loss: 0.4708
Epoch 3/100
508/508 [==============================] - 1s 1ms/step - loss: 0.5180 - val_loss: 0.4577
Epoch 4/100
508/508 [==============================] - 1s 1ms/step - loss: 0.5078 - val_loss: 0.4687
Epoch 5/100
508/508 [==============================] - 1s 1ms/step - loss: 0.4875 - val_loss: 0.4253
Epoch 6/100
508/508 [==============================] - 1s 1ms/step - loss: 0.4814 - val_loss: 0.4192
Epoch 7/100
508/508 [==============================] - 1s 1ms/step - loss: 0.4738 - val_loss: 0.4152
Epoch 8/100
508/508 [==============================] - 1s 1ms/step - loss: 0.4709 - val_loss: 0.4177
Epoch 9/100
508/508 [==============================] - 1s 2ms/step - loss: 0.4635 - val_loss: 0.4130
Epoch 10/100
508/508 [==============================] - 1s 1ms/step - loss: 0.4633 - val_lo

In [90]:
Y_train_pred =model.predict(X_train)
Y_test_pred = model.predict(X_test)
r2_train = r2_score(Y_train, Y_train_pred)
mse_train = mean_squared_error(Y_train, Y_train_pred)

# 测试集上的评估
r2_test = r2_score(Y_test, Y_test_pred)
mse_test = mean_squared_error(Y_test, Y_test_pred)

# 输出结果
print(f"训练集 R^2: {r2_train}")
print(f"训练集 MSE: {mse_train}")
print(f"测试集 R^2: {r2_test}")
print(f"测试集 MSE: {mse_test}")

127/127 [==============================] - 0s 579us/step
训练集 R^2: 0.7271265371245914
训练集 MSE: 0.33811269582365444
测试集 R^2: 0.6951212317761482
测试集 MSE: 0.36352087081936035
